In [13]:
from prometheus_pandas import query
import pandas as pd
from datetime import datetime, timedelta
import pytz
import numpy as np

# Get current UTC time
current_time = datetime.now(pytz.utc)

# Set end time as current time
end = current_time.strftime('%Y-%m-%dT%H:%M:%SZ')

# Set start time as 5 minutes before the current time
start = (current_time - timedelta(minutes=5)).strftime('%Y-%m-%dT%H:%M:%SZ')

p = query.Prometheus('http://r4-infrastructure-prometheus-server.ricplt.svc.cluster.local:80')

# Initialize a DataFrame to store the aggregated results
aggregated_results = pd.DataFrame()

# Loop through each node
for node in range(1, 6):
    node_name = f'node{node}'

    # Query for CPU usage for the specific node
    cpu_query = f'sum by (namespace) (rate(container_cpu_usage_seconds_total{{kubernetes_io_hostname="{node_name}"}}[1m]))'
    df_pod_cpu = p.query_range(cpu_query, start, end, '10s')
    df_pod_cpu['metric'] = 'cpu'

    # Query for Memory usage for the specific node
    mem_query = f'sum by (namespace) (container_memory_working_set_bytes{{kubernetes_io_hostname="{node_name}"}})'
    df_pod_mem = p.query_range(mem_query, start, end, '10s')
    df_pod_mem['metric'] = 'memory'

    # Combine CPU and Memory data
    combined_df = pd.concat([df_pod_cpu, df_pod_mem])

    # Calculate average and standard error for each namespace and metric in this node
    combined_df['avg'] = combined_df.mean(axis=1)
    combined_df['stderr'] = combined_df.std(axis=1) / np.sqrt(combined_df.count(axis=1))
    combined_df['node'] = node_name

    # Append to the aggregated results
    aggregated_results = aggregated_results.append(combined_df)

# Save the aggregated results to a CSV file
aggregated_results.to_csv(f'{start}-out-pod-cpu-mem-avg-stderr.csv')


/tmp/ipykernel_1392231/2356647877.py:39: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  combined_df['avg'] = combined_df.mean(axis=1)
/tmp/ipykernel_1392231/2356647877.py:40: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  combined_df['stderr'] = combined_df.std(axis=1) / np.sqrt(combined_df.count(axis=1))
/tmp/ipykernel_1392231/2356647877.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aggregated_results = aggregated_results.append(combined_df)
/tmp/ipykernel_1392231/2356647877.py:39: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'num